In [1]:
import sys
sys.path.insert(0, "/Users/rodrigo/google_drive/code/")

from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.io import votable, ascii
import json
import numpy as np
from types import SimpleNamespace
from astropy.table import Table, vstack
import ps_api

### SQL query on MAST

In [2]:
def z_cat_query(tl, br):
    string = "select * from catalogRecordRowStore where raMean>{:.2f} and raMean<{:.2f} and decMean>{:.2f} and decMean<{:.2f}".format(br[0], tl[0], br[1], tl[1])
    return string

In [3]:
def sexa_to_deci(c):
    sc = SkyCoord(c, unit=(u.hourangle, u.deg))
    return sc.ra.deg, sc.dec.deg

In [5]:
tl = sexa_to_deci("00:20:46.1004178 +03:52:35.623592")
br = sexa_to_deci("00:18:12.1136067 +03:17:38.998089")
print(z_cat_query(tl, br))

select * from catalogRecordRowStore where raMean>4.55 and raMean<5.19 and decMean>3.29 and decMean<3.88


### Preparing prioritize.py input

Mask A

In [4]:
center = "01:09:42.2955383 +33:04:01.370649"
sc = SkyCoord(center, unit=(u.hourangle, u.deg))
print('hour degree')
print(sc.ra.hour, sexa_to_deci(center)[1])
print('\ndegree degree')
print(*sexa_to_deci(center))

hour degree
1.161748760638889 33.0670474025

degree degree
17.42623140958333 33.0670474025


Mask B: not in the cluster! Will use the same as mask

In [4]:
center = "02:48:03.2952000 -02:15:57.348000"
sc = SkyCoord(center, unit=(u.hourangle, u.deg))
print('hour deg:')
print(sc.ra.hour, sexa_to_deci(center)[1])
print('\ndeg deg:')
print(*sexa_to_deci(center))

hour deg:
2.8009153333333336 -2.26593

deg deg:
42.013729999999995 -2.26593


Cluster center (for axis)

In [14]:
center = "00:19:38.3879576 +03:36:27.749751"
sc = SkyCoord(center, unit=(u.hourangle, u.deg))
# print('hour deg:')
# print(sc.ra.hour, sexa_to_deci(center)[1])
print('\ndeg deg:')
print(*sexa_to_deci(center))


deg deg:
4.909949823333332 3.607708264166667


### Downloading the PS catalog

In [5]:
# cl_pos = ps_api.resolve('RMJ000343.8+100123.8')
cl_pos = sexa_to_deci(center)
cl_pos

(17.42623140958333, 33.0670474025)

In [6]:
# radius = (9 * u.arcmin).to(u.deg).value
radius = (12 * u.arcmin).to(u.deg).value
# radius = .03

In [7]:
# constraints = {'nDetections.gt':1, 'rMeanKronMag.gt':-990}
# constraints = {'rMeanKronMag.gt':-990}
# constraints = {'nDetections.gt':10}
constraints = {'nDetections.gt':1}
# constraints = {}

In [8]:
columns = """objID,raMean,decMean,rMeanKronMag,nDetections""".split(',')
# strip blanks and weed out blank and commented-out values
columns = [x.strip() for x in columns]
columns = [x for x in columns if x and not x.startswith('#')]

In [9]:
results = ps_api.ps1cone(*cl_pos, radius, columns=columns, **constraints)
# try including release="dr2" as an argument

HTTPError: 504 Server Error: Gateway Time-out for url: https://catalogs.mast.stsci.edu/api/v0.1/panstarrs/dr1/mean/metadata

In [13]:
with open('ps_api_res_nDetections1.csv', 'w') as f:
    f.write(results)

### Converting archival redshift file into csv

In [6]:
archival = ascii.read('archivalz.txt', names = ['RA', 'DEC', 'z', 'delta_z'])
archival.write('archivalz.csv', format='ascii.csv')

### Converting output.txt into catalog

In [7]:
io_dir = '1/dsim/'
# io_dir = '2/dsim/'
columns = ['OBJNAME', 'RA', 'DEC', 'EQX', 'MAG', 'band', 'PCODE', 'LIST', 'SEL?', 'PA', 'L1', 'L2']
dtype = []

for column in columns:
    if column=='SEL?':
        dtype.append('i4')
    else:
        dtype.append('<U12')

def read_target_file(filename):
    with open(filename,'r') as targets:
        lines = targets.readlines()

    align = []
    targets = []
    for line in lines:
        line_data = line.strip().split()
        if len(line_data)==9:
            align.append(line_data)
        elif len(line_data)==10:
            targets.append(line_data)
    return align, targets


In [8]:
infilename = "{}output.txt".format(io_dir)
out_selected = "{}selected.csv".format(io_dir)
out_not_selected = "{}not_selected.csv".format(io_dir)
out_align = "{}align.csv".format(io_dir)

In [9]:
align, targets = read_target_file(infilename)

In [10]:
df_targets = Table(np.array(targets), names=columns[:-2], dtype=dtype[:-2])
df_align = Table(np.array(align[:-1]), names=columns[:-3], dtype=dtype[:-3])

In [26]:
df_targets[df_targets['SEL?']==1].write(out_selected, format='ascii.csv', overwrite=True)
df_targets[df_targets['SEL?']==0].write(out_not_selected, format='ascii.csv', overwrite=True)
df_align.write(out_align, format='ascii.csv', overwrite=True)

In [11]:
df_targets[df_targets['SEL?']==1]

OBJNAME,RA,DEC,EQX,MAG,band,PCODE,LIST,SEL?,PA
str12,str12,str12,str12,str12,str12,str12,str12,int32,str12
1791208723,23:20:42.981,+29:07:54.50,2000.0,21.51,r,481,1,1,87
2163387393,23:20:51.914,+29:07:50.51,2000.0,20.89,r,119,1,1,87
1709834434,23:20:41.039,+29:08:11.42,2000.0,21.00,r,1685,1,1,87
1875431500,23:20:45.030,+29:08:32.29,2000.0,21.63,r,1136,1,1,87
2100804805,23:20:50.423,+29:09:12.67,2000.0,21.00,r,5490,1,1,87
2417443004,23:20:58.035,+29:09:07.62,2000.0,21.54,r,1474,1,1,87
2037738950,23:20:48.912,+29:09:55.21,2000.0,21.21,r,2045,1,1,87
1905801192,23:20:45.749,+29:10:01.93,2000.0,21.06,r,4577,1,1,87
1960693808,23:20:47.037,+29:10:10.27,2000.0,20.99,r,4209,1,1,87
